In [24]:
import glob
import os
directory_csv = 'csv_qa_gold'
# csv_files = glob.glob(os.path.join(directory_csv, '**', '*.csv'), recursive=True)
csv_files = ['csv_qa_gold/test_combined.csv']

In [25]:
# read in the csv files in the directory and concatenate them into a single dataframe
import pandas as pd
df_total = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    print(df.shape)
    df_total = pd.concat([df_total, df], axis=0)
print(df_total.shape)
df_total.to_csv('csv_qa_gold/qa_gold.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'csv_qa_gold/test_combined.csv'

In [23]:
df_total.iloc[10:20, :]

,Category,Question,Answer
10,webpages,Was the Scottish terrier a familiar figure at ...,Yes
11,webpages,Is there an official mascot costume of Scotty?,Yes
12,webpages,Who led the effort to make Carnegie Mellon's m...,Susan Bassett and Jennifer Church
13,webpages,Where did Scotty debug as CMU's official mascot?,at a football game
14,webpages,Who did CMU partner with to create the Scotty ...,SME Branding
15,webpages,What month and year was the Mascot Identity Ta...,November 2006
16,webpages,When did Scotty debut as CMU's official mascot?,November 2007
17,webpages,What characteristics does the Scottish terrier...,"determined, thoughtful, strength, power, agili..."
18,webpages,Where did the athletic director of CMU graduat...,Brandeis
19,webpages,When was Carnegie Technical Schools founded?,1900


In [71]:
def generate_answer(question,llm, return_relevant_docs = False):
    answer, relevant_docs = answer_llama_new(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = llm)
    if return_relevant_docs:
        return answer, relevant_docs
    return answer


output_file = 'csv_qa_gold/processed_questions_answers.csv'
# Define the function to process the CSV and add model answers
def generate_answers_and_save(csv_path, llm):
    df = pd.read_csv(csv_path)
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = ''
    for index, row in df.iterrows():
        ModelAnswer = generate_answer(row['Question'], llm)
        df.at[index, 'ModelAnswer'] = ModelAnswer
        df.to_csv(output_file, index=False)
    return df

In [6]:
csv_path = '/mnt/data/questions_answers.csv' 
df_with_model_answers = generate_answers_and_save(csv_path, llm = )
print(df_with_model_answers.shape)
df_with_model_answers.head()

SyntaxError: invalid syntax (1527183504.py, line 2)

# running for all files

In [13]:
type(df['ModelAnswer'][0]),type(df['Answer'][0])
df = df.astype(str)  # Convert columns to string type

In [14]:
# with additional extraction prompt after generation without prompt
from evaluation_csv import total_score_csv
exact_match, f1_score, recall_score = total_score_csv(df['ModelAnswer'], df['Answer'])
print(f'Exact match: {exact_match}')
print(f'F1 score: {f1_score}')
print(f'Recall score: {recall_score}')

Exact match: 0.24087591240875914
F1 score: 0.32063489124366107
Recall score: 0.42024065755817586


In [15]:
for category in df['Category'].unique():
    print(f'Category: {category}')
    df_category = df[df['Category'] == category]
    exact_match, f1_score, recall_score = total_score_csv(df_category['ModelAnswer'], df_category['Answer'])
    print(f'Exact match: {exact_match}')
    print(f'F1 score: {f1_score}')
    print(f'Recall score: {recall_score}')
    print('\n')

Category: webpages
Exact match: 0.24074074074074073
F1 score: 0.40320109831265616
Recall score: 0.591332304526749


Category: Tabular_webpages
Exact match: 0.0
F1 score: 0.0
Recall score: 0.0


Category: tabular_webpages
Exact match: 0.06666666666666667
F1 score: 0.21027471943320888
Recall score: 0.4427350427350427


Category: nan
Exact match: 1.0
F1 score: 1.0
Recall score: 1.0


Category: jsons
Exact match: 0.0
F1 score: 0.0
Recall score: 0.0


Category: json_hard
Exact match: 0.0
F1 score: 0.0
Recall score: 0.0


